
# Parameterization choices for hierarchical models:  centered, non-centered

### Overview

This note expands on the section in Bob Carpenter's most excellent case study
[Hierarchical Partial Pooling for Repeated Binary Trials](https://mc-stan.org/users/documentation/case-studies/pool-binary-trials.html) which introduces the non-centered parameterization of a hierarchical model.

Hierarchical models provide partial pooling of information across parameters
according to group membership.
The hierarchical model provides group-level parameters
which influence the fixed-effects parameters
(individual distributions on the group members).
For datasets where the groups have relatively few members,
MCMC samplers cannot easily explore the resulting sampling density.
The non-centered parameterization mitigates this problem by decoupling
the group-level and fixed-effects parameters in the sampling distribution.

The case study dataset is taken from baseball, consisting of
the number of hits and at-bats for a set of Major League Baseball players.
While individual players have differing batting abilities, they are taken from the population
of MLB baseball players.
Therefore, it makes sense to build a hierarchical model of player ability.

The data consists of `N` observations `y`, where each observation $y_n$ is
the number of successes for ${player}_n$ in `K` trials.
The dataset is small:  18 players ($N = 18$), 45 at-bats ($K = 45$).
The model estimates ${\theta}_n$, each player's chance of success for an at-bat.
($\theta * 1000$ is a player's "batting average".)
It does so by recasting the problem in terms of parameter $\alpha$,
a player's log-odds of success.
The hierarchical model puts a normal prior with group-level parameters
$\mu$ and $\sigma$ on the estimates for parameter $\alpha$,
which pulls the individual player estimates towards the group mean $\mu$.

The log-odds parameterization makes it much easier to expand the model
by adding more fixed effects and other multilevel effects.
The change of success `theta` is computed in the model's
generated quantities block as `inv_logit(alpha)`.

The centered parameterization for a hierarchical model corresponds directly to
the data structure:   the individual-level parameter `alpha` - a player's log-odds of success
is given a prior distribution specified in terms of the group-level parameters:
`alpha ~ normal(mu, sigma)`.
The non-centered parameterization is recommended for hierarchical models
where the groups have relatively few members.
The trick is to decouple `alpha`, `mu`, and `sigma` in the sampling distribution
by reparameterization.
There are two ways to do this reparameterization:

- a non-centered parameterization with standard normal prior on parameter `alpha_std` and auxiliary variable `alpha`.
- a non-centered parameterization using an affine transform on parameter `alpha`.

In this note we show models for each and then plot the result.

### Packages used in this notebook

We use [CmdStanPy](https://mc-stan.org/cmdstanpy) to do the model fitting and plot the results using [plotnine](https://plotnine.readthedocs.io/en/stable/), a ggplot2-like Python package.
Pandas and NumPy are also used for data munging.

In [ ]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from plotnine import *
%matplotlib inline

from cmdstanpy import CmdStanModel

In [ ]:
theme_set(
  theme_grey() + 
  theme(text=element_text(size=10),
        plot_title=element_text(size=14),
        axis_title_x=element_text(size=12),
        axis_title_y=element_text(size=12),
        axis_text_x=element_text(size=8),
        axis_text_y=element_text(size=8)
       )
)

### Baseball Data:  Number of hits in 45 at-bats for 18 MLB players in 1971

In [ ]:
with open('efron-morris-75-data.tsv') as tsv_file:
    df = pd.read_csv("efron-morris-75-data.tsv", sep="\t")
df.style.hide_index().format(precision=3)

In [ ]:
baseball_data = {"N": df.shape[0],
                 "K": df['At-Bats'],
                 "y": df['Hits'],
                 "K_new": df['RemainingAt-Bats'],
                 "y_new": df['SeasonHits']-df['Hits']}

M = 10000  # desired number of draws from the posterior

# ggplot2 x_y plot with axis labels and optional title
def scatter_plot(df, x_lab, y_lab, title=''):
  return (ggplot(df, aes('x', 'y')) +
          geom_point(alpha=0.2) +
          xlab(x_lab) +
          ylab(y_lab) +
          ggtitle(title) +
          theme(figure_size=(8,6)))

## The Model

The model we are interested in is a hierarchical model
with a *normal prior* on the *log odds of success*.
The mathematical model specification is

$$
p(y_n \, | \, K_n, \alpha_n) 
\ = \ \mathsf{Binomial}(y_n \, | \, K_n, \ \mathrm{logit}^{-1}(\alpha_n))
$$

with a simple normal hierarchical prior

$$
p(\alpha_n \, | \, \mu, \sigma)
= \mathsf{Normal}(\alpha_n \, | \, \mu, \sigma).
$$

a weakly informative hyperprior for $\mu$

$$
p(\mu) = \mathsf{Normal}(\mu \, | \, -1, 1),
$$

and a half normal prior on $\sigma$

$$
p(\sigma)
\ = \ 2 \, \mathsf{Normal}(\sigma \, | \, 0, 1)
\ \propto \ \mathsf{Normal}(\sigma \, | \, 0, 1).
$$

### Centered Parameterization

The Stan program `hier-logit-centered.stan` is a straightforward encoding of
a hierarchical model with a normal prior on the log odds of success,
but this is not the optimal way to code this model in Stan, as we will soon demonstrate.

```
parameters {
  real mu;                       // population mean of success log-odds
  real<lower=0> sigma;           // population sd of success log-odds
  vector[N] alpha;               // success log-odds
}
model {
  mu ~ normal(-1, 1);               // hyperprior
  sigma ~ normal(0, 1);             // hyperprior
  alpha ~ normal(mu, sigma);        // prior (hierarchical)
  y ~ binomial_logit(K, alpha);     // likelihood
}
```

The chance of success $\theta$ is computed as a generated quantity.

```
generated quantities {
  vector[N] theta = inv_logit(alpha);
}
```

In CmdStanPy, model fitting is done in two steps:
first instantiate the model object from a Stan program file;
then run the Stan inference algorithm, here the NUTS-HMC sampler,
which returns the inferences.

We instantiate the CmdStanModel object from the Stan program file 'hier-logit-centered.stan'.
By default, CmdStanPy compiles the model on object instantiation, unless there is a corresponding
exe file which has a more recent timestamp than the source file.
The model's `code` method returns the Stan program.

In [ ]:
hier_logit_centered_model = CmdStanModel(stan_file='hier-logit-centered.stan')
print(hier_logit_centered_model.code())

Next run the NUTS-HMC sampler.  By default the sampler runs 4 chains.  The argument `iter_sampling` specifies the *per-chain* number of sampling iterations.  The defaults are 1000 warmup and 1000 sampling iterations per chain,
for a sample containing a total of 4000 draws.  Since $M = 10000$, we override this default.  We specify the random seed for reproducibility.

In [ ]:
fit_centered = hier_logit_centered_model.sample(
    data=baseball_data,
    iter_sampling=int(M/4),
    seed=54321)

The variable `theta` is the per-player chance of success, i.e., `theta * 1000` is a player's batting average.
The estimates range from 0.24 to 0.3, batting averages between 240 and a respectable 300, which is in line
with what we know about major league baseball players.

In [ ]:
fit_centered.summary(sig_figs=3).round(decimals=3).filter(
    regex=r'mu|sigma|theta', axis="index")

The reported Eff values for `sigma` are low and the R_hat value is above 1.   CmdStan's `diagnose` method indicates that this model had problems fitting the data.

In [ ]:
print(fit_centered.diagnose())

### The Funnel

These diagnostics indicate that the sampler failed to fit the data and that the resulting sample is not a sample from
the true posterior.   The reason for this failure is that given the small amount of data,
the sampler cannot properly determine how much of the observed variance in the data is individual-level variance, parameter `alpha`, or group-level variance, parameter `sigma`.
The diagnostics report low ESS and poor R-hat for `sigma`.

Plotting the estimate of `alpha[1]`, the log-odds success for player 1, against `log(sigma)`,
the group-level variance, provide additional evidence of the problem.
This plot shows a clear funnel shape with many draws at the bottom of the neck of the funnel.
This is the reason for the low EFF numbers for `sigma`.  The sampler "gets stuck" at the bottom of the funnel.
The algorithm tries to jump to a new point, but large jumps fall outside of the posterior density, resulting in a divergence.  Small jumps fail to exit the neck of the funnel.

In [ ]:
df_x_y = pd.DataFrame(
    data={'x': fit_centered.stan_variable('alpha')[:,0],
          'y': np.log(fit_centered.stan_variable('sigma'))
         }
)

scatter_plot(df_x_y,
         x_lab = "alpha[1]: player 1 log odds of success",
         y_lab = "log(sigma): log population scale",
         title = "hierarchical vs fixed, centered parameterization")

## The Non-Centered Parameterization 

Instead of a hierarchical prior, 
the non-centered parameterization takes a standard unit normal prior for a new variable,

$$
\alpha^{\mathrm{std}}_n = \frac{\alpha_n - \mu}{\sigma}.
$$

Then we can parameterize in terms of $\alpha^{\mathrm{std}}$, which
has a standard-normal distribution

$$
p(\alpha^{\mathrm{std}}_n) = \mathsf{Normal}(\alpha^{\mathrm{std}}_n \, | \, 0, 1).
$$

We can then define our original $\alpha$ as a derived quantity.

$$
\alpha_n = \mu + \sigma \, \alpha^{\mathrm{std}}_n.
$$

This decouples the sampling distribution
for $\alpha^{\mathrm{std}}$ from $\mu$ and $\sigma$, greatly reducing
their correlation in the posterior.
*The sampler only knows about the model parameters*.
Since the prior on parameter $\alpha$ is not specified in terms of parameters $\mu$ and $\sigma$, the sampler can move more freely along their axes, and therefore explore the posterior more fully.
Although we decouple the parameters, we still need to share information
between the group-level and individual level parameters;
this is done using auxiliary variables, either transformed parameters or
directly in the model block.



###  Non-centered parameterization using a standard normal distribution

Prior to Stan 2.19, a Stan implementation directly encoded the above reparameterization.
This requires 3 changes to the centered parameterization:

- In the parameters block, declaring a parameter `alpha_std` (instead of parameter `alpha`).  This name implies that it will have a standard normal distribution.

- In the transformed parameters block define variable `alpha` as `mu + sigma * alpha_std`.

- In the model block we put a standard normal prior on `alpha_std`, which decouples the sampling distribution of `alpha_std` from `mu` and `sigma`.

The Stan program "hier-logit-nc-std-norm.stan" follows this pattern.
```
data {
  int<lower=0> N; // items
  array[N] int<lower=0> K; // initial trials
  array[N] int<lower=0> y; // initial successes
}
parameters {
  real mu; // population mean of success log-odds
  real<lower=0> sigma; // population sd of success log-odds
  vector[N] alpha_std; // success log-odds (standardized)
}
transformed parameters {
  vector[N] alpha = mu + sigma * alpha_std;
}
model {
  mu ~ normal(-1, 1); // hyperprior
  sigma ~ normal(0, 1); // hyperprior
  alpha_std ~ normal(0, 1); // prior (hierarchical)
  y ~ binomial_logit(K, alpha); // likelihood
}
generated quantities {
  vector[N] theta = inv_logit(alpha);
}
```

###  Non-centered parameterization using an affine transform

Since Stan version 2.19, the Stan language's
[affine transform](https://mc-stan.org/docs/reference-manual/univariate-data-types-and-variable-declarations.html) construct provides a more concise way to do this.
For a real variable, the affine transform $x\mapsto \mu + \sigma * x$ with offset $\mu$ and (positive) multiplier $\sigma$
is specified using a syntax like that used for upper/lower bounds, with keywords <code>offset</code>, <code>multiplier</code>.
Specifying the affine transform in the parameter declaration for 
$\alpha^{\mathrm{std}}$ eliminates the need for intermediate variables
and makes it easier to see the hierarchical structure of the model.

When the parameters to the prior for $\sigma$ are constants, the
normalization for the half-prior (compared to the full prior) is
constant and therefore does not need to be included in the notation.
This only works if the parameters to the density are data or constants;
if they are defined as parameters or as quantities depending on parameters,
then explicit truncation is required.

The Stan program `hier-logit-nc-affine-xform.stan` uses the affine-transform syntax to
specify the non-centered version of the hierarchical model
with a normal prior on the log odds of success.

```
data {
  int<lower=0> N; // items
  array[N] int<lower=0> K; // initial trials
  array[N] int<lower=0> y; // initial successes
}
parameters {
  real mu; // population mean of success log-odds
  real<lower=0> sigma; // population sd of success log-odds
  vector<offset=mu, multiplier=sigma>[N] alpha; // success log-odds (standardized)
}
model {
  mu ~ normal(-1, 1); // hyperprior
  sigma ~ normal(0, 1); // hyperprior
  alpha ~ normal(mu, sigma); // prior (hierarchical)
  y ~ binomial_logit(K, alpha); // likelihood
}
generated quantities {
  vector[N] theta = inv_logit(alpha);
  vector[N] alpha_std = (alpha - mu)/sigma;
}
```

### Fitting the standard normal reparameterization

The model `hier-logit-nc-std-norm.stan` fits the model using parameter `alpha_std`.

*Full disclosure: the choice of random seed '54321' was far from random; this seed allows the sampler to fit the model without divergences.  Other seeds may result in 1 or 2 divergences for a sample of 2500 draws.*

In [ ]:
nc_std_norm_model = CmdStanModel(stan_file='hier-logit-nc-std-norm.stan')
print(nc_std_norm_model.code())

fit_nc_std_norm = nc_std_norm_model.sample(
    data=baseball_data,
    iter_sampling=int(M/4),
    seed=54321)

Again, we check for problems by running CmdStan's `diagnose` method.

In [ ]:
print(fit_nc_std_norm.diagnose())

The estimates for `mu`, `sigma`, `theta` and `alpha` are roughly the same as for the centered parameterization.  The non-centered parameterization results in a much larger effective sample size.

In [ ]:
print("Centered parameterization")
print(fit_centered.summary(sig_figs=3).round(decimals=3).filter(
    ["mu", "sigma",
     "theta[1]", "theta[5]", "theta[10]", "theta[18]",
     "alpha[1]", "alpha[5]", "alpha[10]", "alpha[18]"],
    axis="index"))

print("\nNon-centered parameterization, std_normal reparameterization")
print(fit_nc_std_norm.summary(sig_figs=3).round(decimals=3).filter(
    ["mu", "sigma",
     "theta[1]", "theta[5]", "theta[10]", "theta[18]",
     "alpha[1]", "alpha[5]", "alpha[10]", "alpha[18]"],
    axis="index"))

To consider how the reparameterization is working, we plot the
posterior for the mean and log scale of the hyperprior.
The prior location ($\mu$) and scale ($\sigma$) are coupled in the posterior.

In [ ]:
df_x_y = pd.DataFrame(data={'x': fit_nc_std_norm.stan_variable('mu'),
                            'y': np.log(fit_nc_std_norm.stan_variable('sigma'))})

scatter_plot(df_x_y,
         x_lab = "mu",
         y_lab = "log(sigma)",
         title = "Hierarchical params, standard normal reparameterization")

Now when we plot the sample values for log scale and the first transformed parameter, `alpha_std[1]`,
the range of both the X and Y axis are much wider.  There is a diffuse set of points in the bottom half of the plot, not many points at the bottom of the Y axis.  This indicates that the sampler has been able to properly explore the posterior density and therefore we have a valid sample from the posterior.  As log `sigma` approaches zero the plot has a long right-hand tail.

In [ ]:
df_x_y = pd.DataFrame(
    data={'x': fit_nc_std_norm.stan_variable('alpha_std')[: , 0],
          'y': np.log(fit_nc_std_norm.stan_variable('sigma'))}
)

scatter_plot(df_x_y,
         x_lab = "alpha_std[1]: player 1 log odds of success (transformed)",
         y_lab = "log(sigma): log population scale",
         title = "hierarchical vs fixed param, non-centered parameterization")

We can also plot the value for the generated quantities variable `alpha[1]` against `log(sigma)`
and compare it to the first plot from the centered parameterization.
We recover the funnel shape, but now the Y axis ranges from (-12, 0) instead of (-4, 0).
As above, as log `sigma` approaches zero the plot has a long right-hand tail.

In [ ]:
df_x_y = pd.DataFrame(
    data={'x': fit_nc_std_norm.stan_variable('alpha')[: , 0],
          'y': np.log(fit_nc_std_norm.stan_variable('sigma'))}
)

scatter_plot(df_x_y,
         x_lab = "alpha[1]: player 1 log odds of success",
         y_lab = "log(sigma): log population scale",
         title = "hierarchical param vs generated quantity variable")

We still don't have enough data to determine whether or not the observed variance
is hierarchical or individual-level variance.
The model still provides us with an estimate for `alpha`, a player's log-odds of success at bat.
Critically, because `alpha` is no longer a parameter variable, replaced by `alpha_std`
in the sampling distribution, the sampler can fully explore the posterior.

### Fitting the affine transform parameterization

The model `hier-logit-nc-affine-xform.stan` looks just like the centered parameterization,
with the exception that parameter `alpha` is defined with `<offset = mu, multiplier = sigma>`.

To show that the affine transform reparameterization and the standard normal reparameterization are equivalent
we fit the model to the data and plot the results.


In [ ]:
nc_affine_xform_model = CmdStanModel(stan_file='hier-logit-nc-affine-xform.stan')
print(nc_affine_xform_model.code())

In [ ]:
fit_nc_affine = nc_affine_xform_model.sample(
    data=baseball_data,
    iter_sampling=int(M/4),
    seed=54321)

As usual, we check for problems by running CmdStan's diagnose method.

In [ ]:
print(fit_nc_affine.diagnose())

In [ ]:
print("Centered parameterization")
print(fit_centered.summary(sig_figs=3).round(decimals=3).filter(
    ["mu", "sigma",
     "theta[1]", "theta[5]", "theta[10]", "theta[18]",
     "alpha[1]", "alpha[5]", "alpha[10]", "alpha[18]"],
    axis="index"))

print("\nNon-centered parameterization, std_normal reparameterization")
print(fit_nc_std_norm.summary(sig_figs=3).round(decimals=3).filter(
    ["mu", "sigma",
     "theta[1]", "theta[5]", "theta[10]", "theta[18]",
     "alpha[1]", "alpha[5]", "alpha[10]", "alpha[18]"],
    axis="index"))

print("\nNon-centered parameterization, affine transform reparameterization")
print(fit_nc_affine.summary(sig_figs=3).round(decimals=3).filter(
    ["mu", "sigma",
     "theta[1]", "theta[5]", "theta[10]", "theta[18]",
     "alpha[1]", "alpha[5]", "alpha[10]", "alpha[18]"],
    axis="index"))


We plot the sample values for log scale and the first player ability parameter, `alpha[1]`.
This plot is almost identical to the above plot,
"hierarchical vs generated quantity variable".
Critically, this plot differs from the first plot from the centered parameterization.

In [ ]:
df_x_y = pd.DataFrame(
    data={'x': fit_nc_affine.stan_variable('alpha')[: , 0],
          'y': np.log(fit_nc_affine.stan_variable('sigma'))}
)

scatter_plot(df_x_y,
         x_lab = "alpha[1]: player 1 log odds of success",
         y_lab = "log(sigma): log population scale",
         title = "hierarchical vs fixed param, affine transform")

#### Don't Panic!

You may be asking: *"but this is a funnel plot, isn't this bad?"*

The answer is: *"no!*

Stan reports the parameter estimates on the *constrained* scale,
but it computes on the *unconstrained* scale.
The corresponding unconstrained value for `alpha` is $\frac{\alpha_n - \mu}{\sigma}$.
In the generated quantities block we recover this as variable `alpha_std`.

Plotting `log(sigma)` against `alpha_std[1]` we see the same sampling distribution
as in the standard normal parameterization.

In [ ]:
df_x_y = pd.DataFrame(
    data={'x': fit_nc_affine.stan_variable('alpha_std')[: , 0],
          'y': np.log(fit_nc_affine.stan_variable('sigma'))}
)

scatter_plot(df_x_y,
         x_lab = "alpha_std[1] ==  alpha[1] unconstrained scale",
         y_lab = "log(sigma): log population scale",
         title = "hierarchical vs fixed param, sampling distribution")

Both `hier-logit-nc-std-norm.stan` and `hier-logit-nc-affine-xform.stan` produce essentially
the same results; this is because both models are essentially the same model:
they encode the non-centered parameterization.

In program `hier-logit-nc-std-norm.stan`
we define `alpha` as a transformed parameter and recover `theta`
in the generated quantities block.
```
transformed parameters {
  vector[N] alpha = mu + sigma * alpha_std;
}
...
generated quantities {
  vector[N] theta = inv_logit(mu + sigma * alpha_std);
}
```

In program `hier-logit-nc-affine-xform.stan`
variable `alpha` is a parameter with hierarchical prior `normal(mu, sigma)`.
In the generated quantities block we recover `theta`,
our estimate of a player's chance of success.
Were there a need for it, we would be able to generate variable `alpha_std`
as well.

```
generated quantities {
  vector[N] theta = inv_logit(alpha);
  vector[N] alpha_std = (alpha - mu)/sigma;
}
```

### Discussion

Hierarchical models where the of hierarchical prior is specified in terms
of a location and scale
can be parameterized in one of two ways:  centered or non-centered.
When there are enough per-group observations, the sampler can determine
the amount of group-level variance from the amount of individual-level variance
and the centered parameterization is recommended.
For smaller amounts of per-group observations, the non-centered parameterization
is preferred.

For the non-centered parameterization,
using the affine transform makes it easier to see the hierarchical structure of the model.
When using the affine transform, the sampler computes on the unconstrained scale
reports the parameter value on the constrained scale.
For this reason, using standard normal parameterization may be more computationally efficient,
as it eleminates extra transforms, but for simple models this difference may not be noticeable.

In this note we only consider models with a normal hierarchical prior,
which can be coded either by use of the Stan language's `offset, multiplier` syntax,
or by explicitly introducing a standardized parameter.
Non-normal hierarchical priors are more challenging to reparameterize and are beyond the
scope of this discussion.




